In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# preprocess the data as in deliverable 1
application_df.drop(columns=['EIN','NAME'], inplace=True)
appType_counts = application_df['APPLICATION_TYPE'].value_counts()
classification_counts = application_df['CLASSIFICATION'].value_counts()
replace_application = list(appType_counts[appType_counts < 500].index)
# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
replace_class = list(classification_counts[classification_counts < 1000].index)
# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

application_cat= application_df.columns.drop(['ASK_AMT','IS_SUCCESSFUL'])
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

app_df = application_df[['STATUS','ASK_AMT','IS_SUCCESSFUL']]
app_df = pd.concat([app_df,encode_df],axis=1)
app_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [3]:
y = app_df['IS_SUCCESSFUL'].values
X = app_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/Users/Kelly/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [4]:
# Define the basic neural network model, use only two hidden layers
number_input_features = len(X_train[0])
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=number_input_features))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-04-25 18:44:16.934443: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-25 18:44:17.275200: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/50
804/804 [==============================] - 4s 3ms/step - loss: 0.6787 - accuracy: 0.6237
Epoch 2/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5882 - accuracy: 0.7115
Epoch 3/50
804/804 [==============================] - 7s 8ms/step - loss: 0.5757 - accuracy: 0.7225
Epoch 4/50
804/804 [==============================] - 6s 8ms/step - loss: 0.5714 - accuracy: 0.7238
Epoch 5/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5648 - accuracy: 0.7313
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5630 - accuracy: 0.7291
Epoch 7/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5632 - accuracy: 0.7287
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5579 - accuracy: 0.7355
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5605 - accuracy: 0.7270
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5585 - accuracy: 0.7301

In [6]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create callback that saves the models weight every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True,save_freq=5)

In [8]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50,callbacks=[cp_callback])
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5485 - accuracy: 0.7351

Epoch 00001: saving model to checkpoints/weights.01.hdf5
Epoch 2/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5512 - accuracy: 0.7342

Epoch 00002: saving model to checkpoints/weights.02.hdf5
Epoch 3/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5565 - accuracy: 0.7275

Epoch 00003: saving model to checkpoints/weights.03.hdf5
Epoch 4/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5537 - accuracy: 0.7302

Epoch 00004: saving model to checkpoints/weights.04.hdf5
Epoch 5/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5528 - accuracy: 0.7298

Epoch 00005: saving model to checkpoints/weights.05.hdf5
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5531 - accuracy: 0.7335

Epoch 00006: saving model to checkpoints/weights.06.hdf5
Epoch 7/50
804/804 [==============================] 

In [9]:
# Only got ~72% accuracy for the above and need at least 75% accuracy, so optimize again with new model below
# try using three hidden layers instead of two and 100 epochs instead of 50
# keeping the same number of columns, bins etc to try getting more accuracy with all data
nn_new = tf.keras.models.Sequential()
# First hidden layer
nn_new.add(tf.keras.layers.Dense(units=100, input_dim = number_input_features,activation="relu"))
# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=30,activation="sigmoid"))
# Third hidden layer
nn_new.add(tf.keras.layers.Dense(units=10,activation="sigmoid"))
# output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train new model with 100 epochs
fit_model = nn_new.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 13s 11ms/step - loss: 0.6213 - accuracy: 0.6873

Epoch 00001: saving model to checkpoints/weights.01.hdf5
Epoch 2/100
804/804 [==============================] - 5s 7ms/step - loss: 0.5598 - accuracy: 0.7316

Epoch 00002: saving model to checkpoints/weights.02.hdf5
Epoch 3/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5564 - accuracy: 0.7328

Epoch 00003: saving model to checkpoints/weights.03.hdf5
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5566 - accuracy: 0.7303

Epoch 00004: saving model to checkpoints/weights.04.hdf5
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5530 - accuracy: 0.7304

Epoch 00005: saving model to checkpoints/weights.05.hdf5
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5514 - accuracy: 0.7351

Epoch 00006: saving model to checkpoints/weights.06.hdf5
Epoch 7/100
804/804 [=======================

804/804 [==============================] - 2s 3ms/step - loss: 0.5346 - accuracy: 0.7404

Epoch 00052: saving model to checkpoints/weights.52.hdf5
Epoch 53/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5411 - accuracy: 0.7374

Epoch 00053: saving model to checkpoints/weights.53.hdf5
Epoch 54/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5338 - accuracy: 0.7421

Epoch 00054: saving model to checkpoints/weights.54.hdf5
Epoch 55/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5380 - accuracy: 0.7385

Epoch 00055: saving model to checkpoints/weights.55.hdf5
Epoch 56/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5339 - accuracy: 0.7411

Epoch 00056: saving model to checkpoints/weights.56.hdf5
Epoch 57/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5350 - accuracy: 0.7408

Epoch 00057: saving model to checkpoints/weights.57.hdf5
Epoch 58/100
804/804 [==============================]

In [10]:
# Evaluate new model with scaled data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5529 - accuracy: 0.7259
Loss: 0.5528920888900757, Accuracy: 0.7259474992752075


In [13]:
# got ~73% accuracy with new model, so trying again still with frequency of 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True,save_freq=5)
nn_new = tf.keras.models.Sequential()
# First hidden layer
nn_new.add(tf.keras.layers.Dense(units=13, input_dim = number_input_features,activation="relu"))
# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=8,activation="sigmoid"))
# Third hidden layer
nn_new.add(tf.keras.layers.Dense(units=5,activation="sigmoid"))
# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train new model with 30 epochs
fit_model = nn_new.fit(X_train_scaled, y_train, epochs=30,callbacks=[cp_callback])

Epoch 1/30
  1/804 [..............................] - ETA: 20:19 - loss: 0.8120 - accuracy: 0.3438
Epoch 00001: saving model to checkpoints/weights.01.hdf5
  6/804 [..............................] - ETA: 8s - loss: 0.7745 - accuracy: 0.4069   
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 11/804 [..............................] - ETA: 9s - loss: 0.7632 - accuracy: 0.4277
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 16/804 [..............................] - ETA: 9s - loss: 0.7556 - accuracy: 0.4415
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 21/804 [..............................] - ETA: 11s - loss: 0.7507 - accuracy: 0.4496
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 30/804 [>.............................] - ETA: 11s - loss: 0.7458 - accuracy: 0.4548
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 31/804 [>.............................] - ETA: 12s - loss: 0.7454 - accuracy: 0.4552
Epoch 00001: saving model to checkpoints/we

288/804 [=========>....................] - ETA: 9s - loss: 0.7093 - accuracy: 0.5044
Epoch 00001: saving model to checkpoints/weights.01.hdf5
291/804 [=========>....................] - ETA: 9s - loss: 0.7091 - accuracy: 0.5053
Epoch 00001: saving model to checkpoints/weights.01.hdf5
296/804 [==========>...................] - ETA: 9s - loss: 0.7087 - accuracy: 0.5067
Epoch 00001: saving model to checkpoints/weights.01.hdf5
301/804 [==========>...................] - ETA: 9s - loss: 0.7083 - accuracy: 0.5082
Epoch 00001: saving model to checkpoints/weights.01.hdf5
310/804 [==========>...................] - ETA: 8s - loss: 0.7075 - accuracy: 0.5107
Epoch 00001: saving model to checkpoints/weights.01.hdf5
311/804 [==========>...................] - ETA: 9s - loss: 0.7075 - accuracy: 0.5110
Epoch 00001: saving model to checkpoints/weights.01.hdf5
320/804 [==========>...................] - ETA: 8s - loss: 0.7067 - accuracy: 0.5135
Epoch 00001: saving model to checkpoints/weights.01.hdf5
321/80

576/804 [====================>.........] - ETA: 4s - loss: 0.6889 - accuracy: 0.5663
Epoch 00001: saving model to checkpoints/weights.01.hdf5
581/804 [====================>.........] - ETA: 4s - loss: 0.6886 - accuracy: 0.5670
Epoch 00001: saving model to checkpoints/weights.01.hdf5
586/804 [====================>.........] - ETA: 3s - loss: 0.6882 - accuracy: 0.5678
Epoch 00001: saving model to checkpoints/weights.01.hdf5
595/804 [=====================>........] - ETA: 3s - loss: 0.6877 - accuracy: 0.5691
Epoch 00001: saving model to checkpoints/weights.01.hdf5
600/804 [=====================>........] - ETA: 3s - loss: 0.6874 - accuracy: 0.5699
Epoch 00001: saving model to checkpoints/weights.01.hdf5
601/804 [=====================>........] - ETA: 3s - loss: 0.6873 - accuracy: 0.5700
Epoch 00001: saving model to checkpoints/weights.01.hdf5
609/804 [=====================>........] - ETA: 3s - loss: 0.6868 - accuracy: 0.5712
Epoch 00001: saving model to checkpoints/weights.01.hdf5
611/80

 57/804 [=>............................] - ETA: 16s - loss: 0.5749 - accuracy: 0.7426
Epoch 00002: saving model to checkpoints/weights.02.hdf5
 62/804 [=>............................] - ETA: 15s - loss: 0.5750 - accuracy: 0.7422
Epoch 00002: saving model to checkpoints/weights.02.hdf5
 67/804 [=>............................] - ETA: 15s - loss: 0.5753 - accuracy: 0.7417
Epoch 00002: saving model to checkpoints/weights.02.hdf5
 72/804 [=>............................] - ETA: 15s - loss: 0.5755 - accuracy: 0.7413
Epoch 00002: saving model to checkpoints/weights.02.hdf5
 77/804 [=>............................] - ETA: 14s - loss: 0.5757 - accuracy: 0.7410
Epoch 00002: saving model to checkpoints/weights.02.hdf5
 82/804 [==>...........................] - ETA: 14s - loss: 0.5758 - accuracy: 0.7406
Epoch 00002: saving model to checkpoints/weights.02.hdf5
 90/804 [==>...........................] - ETA: 14s - loss: 0.5763 - accuracy: 0.7399
Epoch 00002: saving model to checkpoints/weights.02.hdf5

343/804 [===========>..................] - ETA: 9s - loss: 0.5805 - accuracy: 0.7317
Epoch 00002: saving model to checkpoints/weights.02.hdf5
347/804 [===========>..................] - ETA: 9s - loss: 0.5805 - accuracy: 0.7317
Epoch 00002: saving model to checkpoints/weights.02.hdf5
356/804 [============>.................] - ETA: 9s - loss: 0.5805 - accuracy: 0.7316
Epoch 00002: saving model to checkpoints/weights.02.hdf5
361/804 [============>.................] - ETA: 9s - loss: 0.5805 - accuracy: 0.7316
Epoch 00002: saving model to checkpoints/weights.02.hdf5
362/804 [============>.................] - ETA: 9s - loss: 0.5805 - accuracy: 0.7316
Epoch 00002: saving model to checkpoints/weights.02.hdf5
367/804 [============>.................] - ETA: 9s - loss: 0.5805 - accuracy: 0.7315
Epoch 00002: saving model to checkpoints/weights.02.hdf5
372/804 [============>.................] - ETA: 8s - loss: 0.5805 - accuracy: 0.7315
Epoch 00002: saving model to checkpoints/weights.02.hdf5
379/80

...

219/804 [=======>......................] - ETA: 8s - loss: 0.5575 - accuracy: 0.7272
Epoch 00029: saving model to checkpoints/weights.29.hdf5
224/804 [=======>......................] - ETA: 8s - loss: 0.5574 - accuracy: 0.7273
Epoch 00029: saving model to checkpoints/weights.29.hdf5
229/804 [=======>......................] - ETA: 8s - loss: 0.5573 - accuracy: 0.7274
Epoch 00029: saving model to checkpoints/weights.29.hdf5
234/804 [=======>......................] - ETA: 8s - loss: 0.5572 - accuracy: 0.7274
Epoch 00029: saving model to checkpoints/weights.29.hdf5
239/804 [=======>......................] - ETA: 8s - loss: 0.5571 - accuracy: 0.7275
Epoch 00029: saving model to checkpoints/weights.29.hdf5
244/804 [========>.....................] - ETA: 8s - loss: 0.5570 - accuracy: 0.7275
Epoch 00029: saving model to checkpoints/weights.29.hdf5
249/804 [========>.....................] - ETA: 8s - loss: 0.5569 - accuracy: 0.7276
Epoch 00029: saving model to checkpoints/weights.29.hdf5
254/80

512/804 [==================>...........] - ETA: 4s - loss: 0.5531 - accuracy: 0.7304
Epoch 00029: saving model to checkpoints/weights.29.hdf5
514/804 [==================>...........] - ETA: 4s - loss: 0.5531 - accuracy: 0.7304
Epoch 00029: saving model to checkpoints/weights.29.hdf5
519/804 [==================>...........] - ETA: 4s - loss: 0.5531 - accuracy: 0.7304
Epoch 00029: saving model to checkpoints/weights.29.hdf5
524/804 [==================>...........] - ETA: 4s - loss: 0.5530 - accuracy: 0.7305
Epoch 00029: saving model to checkpoints/weights.29.hdf5
529/804 [==================>...........] - ETA: 4s - loss: 0.5530 - accuracy: 0.7305
Epoch 00029: saving model to checkpoints/weights.29.hdf5
534/804 [==================>...........] - ETA: 4s - loss: 0.5529 - accuracy: 0.7305
Epoch 00029: saving model to checkpoints/weights.29.hdf5
539/804 [===================>..........] - ETA: 4s - loss: 0.5529 - accuracy: 0.7306
Epoch 00029: saving model to checkpoints/weights.29.hdf5
545/80

799/804 [============================>.] - ETA: 0s - loss: 0.5510 - accuracy: 0.7322
Epoch 00029: saving model to checkpoints/weights.29.hdf5
804/804 [==============================] - 14s 17ms/step - loss: 0.5510 - accuracy: 0.7322
Epoch 30/30
  1/804 [..............................] - ETA: 23s - loss: 0.5108 - accuracy: 0.7812
Epoch 00030: saving model to checkpoints/weights.30.hdf5
  5/804 [..............................] - ETA: 20s - loss: 0.5206 - accuracy: 0.7606
Epoch 00030: saving model to checkpoints/weights.30.hdf5
 10/804 [..............................] - ETA: 14s - loss: 0.5344 - accuracy: 0.7444
Epoch 00030: saving model to checkpoints/weights.30.hdf5
 15/804 [..............................] - ETA: 13s - loss: 0.5414 - accuracy: 0.7350
Epoch 00030: saving model to checkpoints/weights.30.hdf5
 20/804 [..............................] - ETA: 13s - loss: 0.5468 - accuracy: 0.7300
Epoch 00030: saving model to checkpoints/weights.30.hdf5
 25/804 [..............................]

280/804 [=========>....................] - ETA: 8s - loss: 0.5533 - accuracy: 0.7243
Epoch 00030: saving model to checkpoints/weights.30.hdf5
285/804 [=========>....................] - ETA: 8s - loss: 0.5531 - accuracy: 0.7245
Epoch 00030: saving model to checkpoints/weights.30.hdf5
290/804 [=========>....................] - ETA: 8s - loss: 0.5530 - accuracy: 0.7247
Epoch 00030: saving model to checkpoints/weights.30.hdf5
295/804 [==========>...................] - ETA: 8s - loss: 0.5528 - accuracy: 0.7249
Epoch 00030: saving model to checkpoints/weights.30.hdf5
300/804 [==========>...................] - ETA: 8s - loss: 0.5527 - accuracy: 0.7251
Epoch 00030: saving model to checkpoints/weights.30.hdf5
305/804 [==========>...................] - ETA: 8s - loss: 0.5525 - accuracy: 0.7253
Epoch 00030: saving model to checkpoints/weights.30.hdf5
310/804 [==========>...................] - ETA: 8s - loss: 0.5524 - accuracy: 0.7255
Epoch 00030: saving model to checkpoints/weights.30.hdf5
315/80

570/804 [====================>.........] - ETA: 3s - loss: 0.5489 - accuracy: 0.7302
Epoch 00030: saving model to checkpoints/weights.30.hdf5
575/804 [====================>.........] - ETA: 3s - loss: 0.5489 - accuracy: 0.7302
Epoch 00030: saving model to checkpoints/weights.30.hdf5
580/804 [====================>.........] - ETA: 3s - loss: 0.5489 - accuracy: 0.7303
Epoch 00030: saving model to checkpoints/weights.30.hdf5
585/804 [====================>.........] - ETA: 3s - loss: 0.5488 - accuracy: 0.7303
Epoch 00030: saving model to checkpoints/weights.30.hdf5
590/804 [=====================>........] - ETA: 3s - loss: 0.5488 - accuracy: 0.7304
Epoch 00030: saving model to checkpoints/weights.30.hdf5
595/804 [=====================>........] - ETA: 3s - loss: 0.5488 - accuracy: 0.7304
Epoch 00030: saving model to checkpoints/weights.30.hdf5
602/804 [=====================>........] - ETA: 3s - loss: 0.5487 - accuracy: 0.7305
Epoch 00030: saving model to checkpoints/weights.30.hdf5
605/80

In [14]:
# Evaluate new model with scaled data, gives us only 72% again
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5550 - accuracy: 0.7240
Loss: 0.5549949407577515, Accuracy: 0.7239649891853333


In [15]:
# Try changing activation to sigmoid for last hidden layer and frequency is every epoch
cp_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=2, save_weights_only=True,save_freq='epoch')
nn_new = tf.keras.models.Sequential()
# First hidden layer
nn_new.add(tf.keras.layers.Dense(units=8, input_dim = number_input_features,activation="relu"))
# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=5,activation="relu"))
# Third hidden layer
nn_new.add(tf.keras.layers.Dense(units=3,activation="sigmoid"))
# output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train new model with 30 epochs
fit_model = nn_new.fit(X_train_scaled, y_train, epochs=50,callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 6s 7ms/step - loss: 0.7281 - accuracy: 0.5402

Epoch 00001: saving model to checkpoints/weights.01.hdf5
Epoch 2/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5808 - accuracy: 0.7294

Epoch 00002: saving model to checkpoints/weights.02.hdf5
Epoch 3/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5642 - accuracy: 0.7364

Epoch 00003: saving model to checkpoints/weights.03.hdf5
Epoch 4/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5606 - accuracy: 0.7332

Epoch 00004: saving model to checkpoints/weights.04.hdf5
Epoch 5/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5589 - accuracy: 0.7352

Epoch 00005: saving model to checkpoints/weights.05.hdf5
Epoch 6/50
804/804 [==============================] - 5s 6ms/step - loss: 0.5569 - accuracy: 0.7354

Epoch 00006: saving model to checkpoints/weights.06.hdf5
Epoch 7/50
804/804 [==============================] 

In [16]:
# Evaluate accuracy and loss for this model, we get ~73% again
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5529 - accuracy: 0.7271
Loss: 0.55286705493927, Accuracy: 0.7271137237548828


In [17]:
# Try changing activation to all relu for the hidden layers
cp_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True,save_freq='epoch')
nn_new = tf.keras.models.Sequential()
# First hidden layer
nn_new.add(tf.keras.layers.Dense(units=8, input_dim = number_input_features,activation="relu"))
# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=5,activation="relu"))
# Third hidden layer
nn_new.add(tf.keras.layers.Dense(units=3,activation="relu"))
# output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train new model with 100 epochs
fit_model = nn_new.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 6s 5ms/step - loss: 0.6614 - accuracy: 0.6438

Epoch 00001: saving model to checkpoints/weights.01.hdf5
Epoch 2/100
804/804 [==============================] - 4s 6ms/step - loss: 0.5716 - accuracy: 0.7218

Epoch 00002: saving model to checkpoints/weights.02.hdf5
Epoch 3/100
804/804 [==============================] - ETA: 0s - loss: 0.5566 - accuracy: 0.73 - 3s 4ms/step - loss: 0.5566 - accuracy: 0.7321

Epoch 00003: saving model to checkpoints/weights.03.hdf5
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5540 - accuracy: 0.7316

Epoch 00004: saving model to checkpoints/weights.04.hdf5
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5527 - accuracy: 0.7321

Epoch 00005: saving model to checkpoints/weights.05.hdf5
Epoch 6/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5539 - accuracy: 0.7305

Epoch 00006: saving model to checkpoints/weights.06.hdf5
Epoc

Epoch 52/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5481 - accuracy: 0.7324

Epoch 00052: saving model to checkpoints/weights.52.hdf5
Epoch 53/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7327

Epoch 00053: saving model to checkpoints/weights.53.hdf5
Epoch 54/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5410 - accuracy: 0.7395

Epoch 00054: saving model to checkpoints/weights.54.hdf5
Epoch 55/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5441 - accuracy: 0.7364

Epoch 00055: saving model to checkpoints/weights.55.hdf5
Epoch 56/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5464 - accuracy: 0.7299

Epoch 00056: saving model to checkpoints/weights.56.hdf5
Epoch 57/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5453 - accuracy: 0.7362

Epoch 00057: saving model to checkpoints/weights.57.hdf5
Epoch 58/100
804/804 [==================

In [18]:
# Evaluate accuracy and loss for this model
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5563 - accuracy: 0.7256
Loss: 0.5562929511070251, Accuracy: 0.7255976796150208


In [21]:
# Last model to optimize the data for maximum accuracy (highest accuracy so far is 72.7%)
number_input_features = len(X_train_scaled[0])
cp_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True,save_freq=5)
nn_last = tf.keras.models.Sequential()
# First hidden layer
nn_last.add(tf.keras.layers.Dense(units=8, input_dim = number_input_features,activation="relu"))
# Second hidden layer
nn_last.add(tf.keras.layers.Dense(units=5,activation="relu"))
# Third hidden layer
nn_last.add(tf.keras.layers.Dense(units=3,activation="sigmoid"))
# output layer
nn_last.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_last.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train new model with 100 epochs
fit_model = nn_last.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback])
model_loss, model_accuracy = nn_last.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
  1/804 [..............................] - ETA: 22:51 - loss: 0.7490 - accuracy: 0.5312
Epoch 00001: saving model to checkpoints/weights.01.hdf5

Epoch 00001: saving model to checkpoints/weights.01.hdf5
 11/804 [..............................] - ETA: 4s - loss: 0.7625 - accuracy: 0.4924   
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 17/804 [..............................] - ETA: 5s - loss: 0.7740 - accuracy: 0.4723
Epoch 00001: saving model to checkpoints/weights.01.hdf5

Epoch 00001: saving model to checkpoints/weights.01.hdf5
 26/804 [..............................] - ETA: 5s - loss: 0.7760 - accuracy: 0.4656
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 34/804 [>.............................] - ETA: 5s - loss: 0.7743 - accuracy: 0.4654
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 40/804 [>.............................] - ETA: 5s - loss: 0.7729 - accuracy: 0.4654
Epoch 00001: saving model to checkpoints/weights.01.hdf5

Epoch 00001: 

306/804 [==========>...................] - ETA: 6s - loss: 0.7375 - accuracy: 0.4688
Epoch 00001: saving model to checkpoints/weights.01.hdf5
311/804 [==========>...................] - ETA: 6s - loss: 0.7371 - accuracy: 0.4691
Epoch 00001: saving model to checkpoints/weights.01.hdf5
319/804 [==========>...................] - ETA: 6s - loss: 0.7364 - accuracy: 0.4697
Epoch 00001: saving model to checkpoints/weights.01.hdf5

Epoch 00001: saving model to checkpoints/weights.01.hdf5
326/804 [===========>..................] - ETA: 6s - loss: 0.7357 - accuracy: 0.4702
Epoch 00001: saving model to checkpoints/weights.01.hdf5
335/804 [===========>..................] - ETA: 6s - loss: 0.7349 - accuracy: 0.4711
Epoch 00001: saving model to checkpoints/weights.01.hdf5
336/804 [===========>..................] - ETA: 6s - loss: 0.7349 - accuracy: 0.4712
Epoch 00001: saving model to checkpoints/weights.01.hdf5
341/804 [===========>..................] - ETA: 6s - loss: 0.7344 - accuracy: 0.4717
Epoch

636/804 [======================>.......] - ETA: 1s - loss: 0.7130 - accuracy: 0.5129
Epoch 00001: saving model to checkpoints/weights.01.hdf5
641/804 [======================>.......] - ETA: 1s - loss: 0.7127 - accuracy: 0.5136
Epoch 00001: saving model to checkpoints/weights.01.hdf5

Epoch 00001: saving model to checkpoints/weights.01.hdf5
651/804 [=======================>......] - ETA: 1s - loss: 0.7120 - accuracy: 0.5149
Epoch 00001: saving model to checkpoints/weights.01.hdf5
656/804 [=======================>......] - ETA: 1s - loss: 0.7117 - accuracy: 0.5155
Epoch 00001: saving model to checkpoints/weights.01.hdf5

Epoch 00001: saving model to checkpoints/weights.01.hdf5
666/804 [=======================>......] - ETA: 1s - loss: 0.7111 - accuracy: 0.5168
Epoch 00001: saving model to checkpoints/weights.01.hdf5
675/804 [========================>.....] - ETA: 1s - loss: 0.7106 - accuracy: 0.5180
Epoch 00001: saving model to checkpoints/weights.01.hdf5
677/804 [=======================

163/804 [=====>........................] - ETA: 7s - loss: 0.6078 - accuracy: 0.7267
Epoch 00002: saving model to checkpoints/weights.02.hdf5
167/804 [=====>........................] - ETA: 7s - loss: 0.6077 - accuracy: 0.7268
Epoch 00002: saving model to checkpoints/weights.02.hdf5
172/804 [=====>........................] - ETA: 7s - loss: 0.6075 - accuracy: 0.7269
Epoch 00002: saving model to checkpoints/weights.02.hdf5
177/804 [=====>........................] - ETA: 7s - loss: 0.6075 - accuracy: 0.7270
Epoch 00002: saving model to checkpoints/weights.02.hdf5
182/804 [=====>........................] - ETA: 7s - loss: 0.6074 - accuracy: 0.7270
Epoch 00002: saving model to checkpoints/weights.02.hdf5

Epoch 00002: saving model to checkpoints/weights.02.hdf5
192/804 [======>.......................] - ETA: 6s - loss: 0.6072 - accuracy: 0.7271
Epoch 00002: saving model to checkpoints/weights.02.hdf5
200/804 [======>.......................] - ETA: 6s - loss: 0.6071 - accuracy: 0.7272
Epoch

503/804 [=================>............] - ETA: 2s - loss: 0.6033 - accuracy: 0.7268
Epoch 00002: saving model to checkpoints/weights.02.hdf5
511/804 [==================>...........] - ETA: 2s - loss: 0.6031 - accuracy: 0.7268
Epoch 00002: saving model to checkpoints/weights.02.hdf5
515/804 [==================>...........] - ETA: 2s - loss: 0.6031 - accuracy: 0.7268
Epoch 00002: saving model to checkpoints/weights.02.hdf5

Epoch 00002: saving model to checkpoints/weights.02.hdf5
522/804 [==================>...........] - ETA: 2s - loss: 0.6030 - accuracy: 0.7268
Epoch 00002: saving model to checkpoints/weights.02.hdf5
530/804 [==================>...........] - ETA: 2s - loss: 0.6029 - accuracy: 0.7267
Epoch 00002: saving model to checkpoints/weights.02.hdf5

Epoch 00002: saving model to checkpoints/weights.02.hdf5
537/804 [===================>..........] - ETA: 2s - loss: 0.6028 - accuracy: 0.7267
Epoch 00002: saving model to checkpoints/weights.02.hdf5

Epoch 00002: saving model to ch

...

464/804 [================>.............] - ETA: 4s - loss: 0.5422 - accuracy: 0.7389
Epoch 00099: saving model to checkpoints/weights.99.hdf5
469/804 [================>.............] - ETA: 4s - loss: 0.5422 - accuracy: 0.7389
Epoch 00099: saving model to checkpoints/weights.99.hdf5
474/804 [================>.............] - ETA: 4s - loss: 0.5422 - accuracy: 0.7389
Epoch 00099: saving model to checkpoints/weights.99.hdf5
479/804 [================>.............] - ETA: 4s - loss: 0.5422 - accuracy: 0.7389
Epoch 00099: saving model to checkpoints/weights.99.hdf5
485/804 [=================>............] - ETA: 4s - loss: 0.5422 - accuracy: 0.7389
Epoch 00099: saving model to checkpoints/weights.99.hdf5
490/804 [=================>............] - ETA: 3s - loss: 0.5423 - accuracy: 0.7389
Epoch 00099: saving model to checkpoints/weights.99.hdf5
494/804 [=================>............] - ETA: 3s - loss: 0.5423 - accuracy: 0.7389
Epoch 00099: saving model to checkpoints/weights.99.hdf5
499/80

754/804 [===========================>..] - ETA: 0s - loss: 0.5435 - accuracy: 0.7382
Epoch 00099: saving model to checkpoints/weights.99.hdf5
762/804 [===========================>..] - ETA: 0s - loss: 0.5435 - accuracy: 0.7382
Epoch 00099: saving model to checkpoints/weights.99.hdf5
764/804 [===========================>..] - ETA: 0s - loss: 0.5435 - accuracy: 0.7382
Epoch 00099: saving model to checkpoints/weights.99.hdf5
769/804 [===========================>..] - ETA: 0s - loss: 0.5436 - accuracy: 0.7382
Epoch 00099: saving model to checkpoints/weights.99.hdf5
774/804 [===========================>..] - ETA: 0s - loss: 0.5436 - accuracy: 0.7382
Epoch 00099: saving model to checkpoints/weights.99.hdf5
779/804 [============================>.] - ETA: 0s - loss: 0.5436 - accuracy: 0.7381
Epoch 00099: saving model to checkpoints/weights.99.hdf5
784/804 [============================>.] - ETA: 0s - loss: 0.5436 - accuracy: 0.7381
Epoch 00099: saving model to checkpoints/weights.99.hdf5
789/80

230/804 [=======>......................] - ETA: 8s - loss: 0.5450 - accuracy: 0.7349
Epoch 00100: saving model to checkpoints/weights.100.hdf5
235/804 [=======>......................] - ETA: 8s - loss: 0.5450 - accuracy: 0.7349
Epoch 00100: saving model to checkpoints/weights.100.hdf5
240/804 [=======>......................] - ETA: 8s - loss: 0.5450 - accuracy: 0.7349
Epoch 00100: saving model to checkpoints/weights.100.hdf5
245/804 [========>.....................] - ETA: 8s - loss: 0.5450 - accuracy: 0.7350
Epoch 00100: saving model to checkpoints/weights.100.hdf5
250/804 [========>.....................] - ETA: 8s - loss: 0.5450 - accuracy: 0.7350
Epoch 00100: saving model to checkpoints/weights.100.hdf5
255/804 [========>.....................] - ETA: 8s - loss: 0.5451 - accuracy: 0.7350
Epoch 00100: saving model to checkpoints/weights.100.hdf5
260/804 [========>.....................] - ETA: 8s - loss: 0.5451 - accuracy: 0.7350
Epoch 00100: saving model to checkpoints/weights.100.hdf5

515/804 [==================>...........] - ETA: 4s - loss: 0.5462 - accuracy: 0.7345
Epoch 00100: saving model to checkpoints/weights.100.hdf5
524/804 [==================>...........] - ETA: 4s - loss: 0.5463 - accuracy: 0.7345
Epoch 00100: saving model to checkpoints/weights.100.hdf5
528/804 [==================>...........] - ETA: 4s - loss: 0.5463 - accuracy: 0.7345
Epoch 00100: saving model to checkpoints/weights.100.hdf5
532/804 [==================>...........] - ETA: 4s - loss: 0.5463 - accuracy: 0.7345
Epoch 00100: saving model to checkpoints/weights.100.hdf5
535/804 [==================>...........] - ETA: 4s - loss: 0.5463 - accuracy: 0.7345
Epoch 00100: saving model to checkpoints/weights.100.hdf5
540/804 [===================>..........] - ETA: 3s - loss: 0.5463 - accuracy: 0.7345
Epoch 00100: saving model to checkpoints/weights.100.hdf5
545/804 [===================>..........] - ETA: 3s - loss: 0.5463 - accuracy: 0.7345
Epoch 00100: saving model to checkpoints/weights.100.hdf5

800/804 [============================>.] - ETA: 0s - loss: 0.5464 - accuracy: 0.7347
Epoch 00100: saving model to checkpoints/weights.100.hdf5
804/804 [==============================] - 13s 16ms/step - loss: 0.5464 - accuracy: 0.7347
268/268 - 1s - loss: 0.5532 - accuracy: 0.7258
Loss: 0.5532086491584778, Accuracy: 0.7258309125900269


In [22]:
model_loss, model_accuracy = nn_new.evaluate(X_train_scaled, y_train, verbose=2)
model_loss2, model_accuracy2 = nn_last.evaluate(X_train_scaled,y_train,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Loss: {model_loss2}, Accuracy: {model_accuracy2}")

804/804 - 1s - loss: 0.5429 - accuracy: 0.7370
804/804 - 1s - loss: 0.5456 - accuracy: 0.7365
Loss: 0.542881429195404, Accuracy: 0.7370160222053528
Loss: 0.5455591678619385, Accuracy: 0.7365494966506958


In [23]:
# The best accuracy was always between ~72% to ~74% as seen from the models above, so we will save the last one into h5

In [24]:
nn_new.load_weights("checkpoints/weights.100.hdf5")
# Export model
nn_new.save("AlphabetSoupCharity_Optimization.h5")
# Import model to a new object
nn_imported = tf.keras.models.load_model('AlphabetSoupCharity.h5')

In [25]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5546 - accuracy: 0.7264
Loss: 0.5545734763145447, Accuracy: 0.7264139652252197
